In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
save_path = './save/dialoguesum_4'
model_path = os.path.join(save_path, 'pytorch.bin')

dialoguesum_path = './data/dialogsum/'
clean_path = os.path.join(dialoguesum_path, 'clean_data')
test_path = os.path.join(clean_path, 'test.json')

In [ ]:
from hf_train_bart import Bart
model = Bart()

In [3]:
model.args.oracle_functurn_context = False
model.args.ctrl_nb_summary_sent = 0
model.args.source_max_len = 512
model.args.target_max_len = 50
model.args.eval_batch_size = 8
model.args.no_repeat_ngram_size = 0
model.args.beam = 4
model.arg.dev_file_path = './dta/new_dialogsum/clean_data/'

In [17]:
scores = model.evaluate(source="test", dump_pred=model.args.dump_pred)


./data/dialogsum/clean_data/test.json 837
examples[0].ID test_0#0
examples[0].context <s> <hl> <s> #Person1#: ms . dawson , i need you to take a dictation for me . <s> #Person2#: yes , sir . <s> #Person1#: this should go out as an intra-office memorandum to all employees by this afternoon . are you ready ? <s> #Person2#: yes , sir . go ahead . <s> #Person1#: attention all staff . effective immediately , all office communications are restricted to email correspondence and official memos . the use of instant message programs by employees during working hours is strictly prohibited . <s> #Person2#: sir , does this apply to intra-office communications only ? or will also restrict external communications ? <s> #Person1#: should apply to all communications , not only in this office between employees , but also any outside communications . <s> #Person2#: but sir , many employees use instant messaging to communicate with their clients . <s> #Person1#: they will just have to change their commu

Examples: 100%|███████████████████████████████████████████████████| 837/837 [00:00<00:00, 868.97it/s]


[INFO] max_target_len 207
[INFO] Testing file_path: ./data/dialogsum/clean_data/test.json


Generating:   0%|                                                            | 0/105 [00:00<?, ?it/s]/home/tnguyen/miniconda/envs/coref/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
Generating: 100%|██████████████████████████████████████████████████| 105/105 [01:56<00:00,  1.11s/it]





rouge-1:	P: 13.4676	R: 14.1793	F1: 13.2597
rouge-2:	P: 6.1735	R: 6.6761	F1: 6.1359
rouge-3:	P: 3.5968	R: 4.0101	F1: 3.6105
rouge-4:	P: 2.0095	R: 2.3155	F1: 2.0354
rouge-l:	P: 13.1440	R: 13.7600	F1: 13.0556
rouge-w:	P: 10.1197	R: 6.4650	F1: 7.5375





In [4]:
import json
summarys = {}
f_gold = json.load(open("./data/dialogsum/clean_data/test.json", "r"))
for di, data in enumerate(f_gold):
    summarys[data["id"]] = data["summary"].strip().replace("\n", " ").replace("\015", "")

In [5]:
f_pred_bart_single = json.load(open(os.path.join(model.args.output_dir, "test.pred.summary{}.json".format(model.args.add_name)), "r"))
update_pred = {}
for item_id, item in f_pred_bart_single.items():
    ID, turn = item_id.rsplit("#", 1)
    if ID not in update_pred.keys():
        update_pred[ID] = {}
    update_pred[ID][turn] = item


In [6]:
from hf_train_bart import SequenceMatcher

final_full_pred = {}
for ID, value in update_pred.items():
    final_full_pred[ID] = []
    for t in range(len(value)):
        if t == 0:
            final_full_pred[ID].append(value[str(t)])
        else:
            similar_score_max = max([SequenceMatcher(None, s, value[str(t)]).ratio() for s in final_full_pred[ID]])
            if similar_score_max < 0.75:
                final_full_pred[ID].append(value[str(t)])


In [25]:
test_summary = []
test_prediction = []
for ID, value in final_full_pred.items():
    test_summary.append(summarys[ID])
    test_prediction.append(" ".join(value))

In [27]:
import rouge

evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                        max_n=4,
                        limit_length=True,
                        length_limit=100,
                        length_limit_type='words',
                        apply_avg=False,
                        apply_best=True,
                        alpha=0.5, # Default F1_score
                        weight_factor=1.2,
                        stemming=True)

scores = evaluator.get_scores(test_prediction, test_summary)

In [28]:
for metric, results in sorted(scores.items(), key=lambda x: x[0]):
    print('{}:\t{}: {:5.4f}\t{}: {:5.4f}\t{}: {:5.4f}'.format(metric, 'P', 100.0 * results['p'], 'R', 100.0 * results['r'], 'F1', 100.0 * results['f']))
    

rouge-1:	P: 48.7539	R: 48.2481	F1: 46.7462
rouge-2:	P: 22.2641	R: 22.2353	F1: 21.3931
rouge-3:	P: 13.0559	R: 13.1686	F1: 12.5566
rouge-4:	P: 7.6753	R: 7.8181	F1: 7.3817
rouge-l:	P: 47.0230	R: 46.5826	F1: 45.5739
rouge-w:	P: 35.2023	R: 20.0643	F1: 24.4631


In [32]:
for i in range(5):
    print(test_prediction[i])
    print(test_summary[i])
    print('\n\n')

#Person1# asks Ms. Dawson to take a dictation for #Person1# and asks her to prepare an intra-office memorandum to all employees by this afternoon.
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.



#Person2# got stuck in traffic again. #Person1# advises #Person2# to start biking.
#Person2# arrives late because of traffic jam. #Person1# persuades #Person2# to use public transportations to keep healthy and to protect the environment.



#Person1# tells Kate that Masha and Hero are getting divorced. #Person1# thinks they are the perfect couple and they will divorce early.
#Person1# tells Kate that Masha and Hero get divorced. Kate is surprised because she thought they are perfect couple.



#Person1# wishes Brian a happy birthday. They will have a drink together to celebrate.
#Person1# and Brian are at the birthday party of Brian. Brian thinks #Person1# looks great and is po